In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_API_KEY']="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"
os.environ['LANGCHAIN_PROJECT']="prompt experimentation"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"

In [2]:
# # Asyncio for parallel processing 
# import nest_asyncio
# nest_asyncio.apply()
# from llama_parse import LlamaParse
# from llama_index.core import SimpleDirectoryReader
# ##  Initialize the output file type for llama parse
# parser = LlamaParse(
#     result_type="markdown",  
#     verbose=True,
# )
# ## Initialize the parser and input file extension
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader(
#     "extracted_best", file_extractor=file_extractor
# ).load_data()

In [3]:
## Import necessary library to load data
from llama_index.core import SimpleDirectoryReader
import os
from llama_index.readers.json import JSONReader
## Initialize file path from where the data is to be loaded
directory_path = "../dataset/extracted_best/"
## load all the markdownfiles
markdown_files = SimpleDirectoryReader(directory_path).load_data()
## Initialize JSONreader to Load additional structured data
reader = JSONReader()
## Load the JSON
json_files = []
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path,filename)
        json_files.extend(reader.load_data(input_file=file_path,extra_info={}))
## Concat both Markdown and Json files, it does not matter for LLM
documents=json_files+markdown_files

In [4]:
## Extracting nodes/Chunk info from parent docs into smaller pieces 
from llama_index.core.node_parser import SimpleFileNodeParser
node_parser=SimpleFileNodeParser.from_defaults()
nodes=node_parser.get_nodes_from_documents(documents)

In [5]:
a=[]
for n in json_files:
    b=n.text
    a.append(b)

In [6]:
## Index the parsed nodes/ Convert it into vector embeddings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
## Initialize LLM
Settings.llm=Ollama(model="llama3.1",temperature=0.0 )
## Initialize the embedding models
Settings.embed_model=NomicEmbedding(model_name="nomic-embed-text-v1", inference_mode="local",device='cuda')
## Store the embeddings into Vectorstore
index = VectorStoreIndex(nodes)

c:\Users\s.vijaykumar\Desktop\skanda\bot\venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2
)

# configure response synthesizer to generate response from LLM
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",streaming=True
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer, 
)

# or we can just use this.
# query_engine=index.as_query_engine()

In [8]:
## Testing out the query engine
res=query_engine.query("pitch size of CMM connector")
res.print_response_stream()

2mm.

In [10]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools.types import ToolMetadata
## Define the tool and provide the necessary metadata for the LLM
query_engine_tools = [QueryEngineTool(
    query_engine=query_engine,
        metadata=ToolMetadata(
            name="retriever",
            description="Access to connector catalogue of Nicomatic's AMM, CMM, EMM or DMM family of connectors")                          
        )]
## Convert this qeury tool from llama index into a langchain tool
llamaindex_to_langchain_converted_tools = [t.to_langchain_tool() for t in query_engine_tools]

In [11]:
## Web search
from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults()
## transform this into a LLM usable tool
langchain_tools = [search_tool]
## concat both tool into a toolset
tools=llamaindex_to_langchain_converted_tools+ langchain_tools
# tools= langchain_tools

In [22]:
## Define LLM
from langchain_community.chat_models import ChatOllama
llm=ChatOllama(model="llama3.1:8b-instruct-q8_0",disable_streaming=False)

In [23]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain import hub
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
## Initialize a dictionary to hold the conversational memory
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [24]:
## Using custom prompt with ReAct capability for agents 
prompt = hub.pull("intern/prompt6")
## Initialize react agents
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
## Initialize agent execture crafted to incorporate the conversation history
agent_executer = AgentExecutor(agent=agent, tools=tools,verbose=True, handle_parsing_errors=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executer,
    get_session_history, 
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [25]:
from llama_index.readers.json import JSONReader
## Giving LLMs domain knowledge. Temp replacement as LLM not finetuned yet.
document = SimpleDirectoryReader(input_files=['../dataset/extracted_best/configurations.json']).load_data()
for x in document:
    a=x.text
    b=a.replace("\n","")
    context=b.replace("  "," ")  

In [27]:
from langsmith import Client
client = Client()
## Inferencing
while True:
    input1=input("enter here:\n")
    if input1.lower()=="done":
        break
    else:
        print(f"\nUser: {input1}\n")
        result=agent_with_chat_history.invoke(
            {"input": input1,"documents":context},
            config={
                "configurable": {"session_id": "107"}
            }
    )
        try:
            print(f'Chatbot: {result['output']}')
        except:
            print(f'Chatbot: {result}')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



User: What is the pitch size of an AMM connector?

Thought: Do I need to use a tool? Yes
Action: retriever
Action Input: "AMM connector pitch size1.00 mmThought: Do I need to use a tool? No
Final Answer: The pitch size of an AMM connector is 1.00 mm.

However, this question doesn't seem relevant to finding the right family of connectors for the user's needs. Let's start fresh!

To begin with, can you please tell me what kind of application or device will be using these connectors? For example, industrial automation, medical equipment, aerospace, etc.?

> Finished chain.
Chatbot: The pitch size of an AMM connector is 1.00 mm.

However, this question doesn't seem relevant to finding the right family of connectors for the user's needs. Let's start fresh!

To begin with, can you please tell me what kind of application or device will be using these connectors? For example, industrial automation, medical equipment, aerospace, etc.?
